In [ ]:
!wget https://zenodo.org/records/7812323/files/training_data.zip?download=1

--2024-04-19 22:29:35--  https://zenodo.org/records/7812323/files/training_data.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10927391529 (10G) [application/octet-stream]
Saving to: ‘training_data.zip?download=1’

training_data.zip?d 100%[===================>]  10.18G  29.2MB/s    in 6m 10s  

2024-04-19 22:35:46 (28.1 MB/s) - ‘training_data.zip?download=1’ saved [10927391529/10927391529]



In [ ]:
!unzip training_data.zip

In [ ]:
import json
with open("/content/training_data/quadrant-enumeration-disease/train_quadrant_enumeration_disease.json", 'r') as f:
  stats = json.load(f)

stats.keys()

In [ ]:
# stats['images'][0]['file_name'] # gives the file address
stats['annotations'][1]['bbox'] # For fetching the Bboxes for one sample

[{'iscrowd': 0,
  'image_id': 1,
  'bbox': [542.0, 698.0, 220.0, 271.0],
  'segmentation': [[621,
    703,
    573,
    744,
    542,
    885,
    580,
    945,
    650,
    969,
    711,
    883,
    762,
    807,
    748,
    741,
    649,
    698]],
  'id': 1,
  'area': 39683,
  'category_id_1': 3,
  'category_id_2': 7,
  'category_id_3': 0},
 {'iscrowd': 0,
  'image_id': 1,
  'bbox': [1952.0, 693.0, 177.0, 270.0],
  'segmentation': [[2045,
    693,
    2109,
    734,
    2129,
    915,
    2047,
    963,
    2009,
    909,
    1973,
    851,
    1955,
    782,
    1952,
    730,
    2000,
    702]],
  'id': 2,
  'area': 33301,
  'category_id_1': 2,
  'category_id_2': 7,
  'category_id_3': 0},
 {'iscrowd': 0,
  'image_id': 1,
  'bbox': [675.0, 708.0, 243.0, 300.0],
  'segmentation': [[784,
    711,
    754,
    746,
    737,
    821,
    678,
    916,
    675,
    986,
    690,
    1003,
    727,
    1008,
    787,
    998,
    851,
    925,
    885,
    857,
    918,
    807,
    9



---
**Skeleton of the haar extraction function**


In [ ]:
# @title
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import cv2

mpl.rcParams['image.cmap'] = 'Greys'


img = [plt.imread("/content/training_data/quadrant/xrays/train_2.png")] # Reading one example image
# cA, (cH, cV, cD) = wav(img, 1)

In [ ]:
import pywt
import pywt.data
import torch
from pywt import dwt2, idwt2

In [ ]:
# Defining a helper function which calculates the haar components for the image
def wav(img, flg):
      if(flg == 1): # A flag for the first pass to take care of converting to grey scale
        image=img[0]
        image2= 0.2989 * image[:,:,0] + 0.5870 * image[:,:,1] + 0.1140 * image[:,:,2]
      else:
        image2 = img # If not in RGB
      coeffs2 = pywt.dwt2(image2, 'haar')
      return coeffs2

In [ ]:
# The main function to extract the images
def extract(img, bbox, R, weights):
  # Normalizing weights
  weights = weights / np.sum(weights)

  # Initializing variables
  prev_img = img
  flg = 1
  prev_bbox = bbox.copy()
  new_bbox = np.zeros(4)
  # out = np.zeros(4)

  # The for loop for multiple iterations of HAAR compressions (3 seems to work best?)
  for j in range(R):
    cA, (cH, cV, cD)= wav(prev_img, flg) # Getting the coeff
    flg = 0 # Setting flag for first pass

    # Taking weighted average over all the outputs for HAAR wavelet
    out = np.zeros_like(cA)
    out += cA * weights[0]
    out += cH * weights[1]
    out += cV * weights[2]
    out += cD * weights[3]

    # out = cv2.merge((cA, cH, cV, cD))
    prev_img = out
    # print(f"This is the {j}'th iteration. Where the output from haar has {out.shape} shape") # In case want to check shape (For Bboxes)

    # Taking care of the Bboxes
    for i in range(4): # Will need to change the number of iteration for tooth (as it is not same for all the samples)
      new_bbox[i] = prev_bbox[i]/2
      # print("Prev", prev_bbox[i], "New", new_bbox[i], "\n") # In case want to check what is happening
      prev_bbox[i] = new_bbox[i]
  return out, new_bbox

In [ ]:
# Deez weights 0.3473684210526316
# Deez weights 0.3474 Which we use

The cell below is to check what is the best weight to assign to the haar components for best representation.

In [ ]:
for Wtl in np.linspace(0, 1, 20, endpoint = True):
  # Wtl = 0.9
  wtEdges = (1 - Wtl)/3
  weights = [Wtl] + [wtEdges]*3
  # print(img[0].shape)
  feat, new_bbox = extract(img, stats['annotations'][1]['bbox'], 3, weights)
  feat = feat/feat.max()

  recon = cv2.resize(feat, dsize = img[0].shape[:2][::-1], interpolation = cv2.INTER_CUBIC) # Resizing to compare with the old image (Here interpolation will not yield the exact results)
                                                                                            # but to reconstruct the image we will need all the haar components seperately so second best
                                                                                            # option is to do this
  # Checking the distance between the images after taking a greyscale
  image=img[0]
  image2= 0.2989 * image[:,:,0] + 0.5870 * image[:,:,1] + 0.1140 * image[:,:,2]
  print(Wtl, ((recon - image2)**2).sum())
  # plt.imshow(feat[:, :]); plt.title(f"Wtl = {Wtl}"); plt.show();

NameError: name 'np' is not defined

In [ ]:
Weigths = 0.3474



---

